### Membaca data

In [1]:
import pandas as pd
import numpy
numpy.set_printoptions(suppress=True)

In [2]:
# Importing data
data = pd.read_csv('clean_data.csv')         # data primer    
del data['Unnamed: 0']
data

,1,10,100,101,102,103,104,105,106,107,...,91,92,93,94,95,96,97,98,99,radiant_win
0,0,-1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
621059,0,0,0,0,-1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
621060,0,0,0,0,0,0,0,0,1,0,...,0,0,-1,0,0,0,0,0,0,0
621061,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
621062,0,0,-1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [3]:
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

### Model

In [4]:
import time

from tensorflow.keras import Sequential
from tensorflow.keras.layers import InputLayer, Dense, Dropout
from tensorflow.keras.optimizers import SGD
from keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.constraints import MaxNorm

In [5]:
# Set the output and hidden layer sizes
input_neuron= 121
# hidden_neuron = 300
output_neuron = 1

learning_rate = 0.001
momentum = 0.9
init_mode = 'random_uniform'
activation='gelu'
dropout_rate = 0.4
weight_constraint = 3

In [6]:
# Function to create model, required for KerasClassifier
              
def create_model(hidden_neuron = 1):
    # create model
    model = Sequential()
    model.add(InputLayer(input_shape=(input_neuron,)))
    model.add(Dense(hidden_neuron, kernel_initializer=init_mode, activation=activation, 
                    kernel_constraint=MaxNorm(weight_constraint)))
    model.add(Dropout(dropout_rate))
    model.add(Dense(output_neuron, kernel_initializer=init_mode, activation='sigmoid'))
    
    # Choose the optimizer and the loss function
    optimizer = SGD(learning_rate=learning_rate, momentum=momentum)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [7]:
batch_size = 80
epochs = 10

In [8]:
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

# create model
model = KerasClassifier(build_fn=create_model, epochs=epochs, batch_size=batch_size, verbose=0)

In [9]:
from sklearn.model_selection import RandomizedSearchCV

start_time = time.time()

# define the grid search parameters
# optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
# learning_rate = [0.001, 0.01, 0.1, 0.2, 0.3]
# momentum = [0.0, 0.2, 0.4, 0.6, 0.8, 0.9]
# init_mode = [
#     'uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform',
#     'identity', 'lecun_normal', 'ones', 'orthogonal', 'random_normal', 'random_uniform', 'truncated_normal',
#     'variance_scaling', 
# ]
# activation = [
#     'softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear', 'elu', 'exponential',
#     'gelu', 'hard_sigmoid', 'sigmoid', 'swish', 
# ]
# weight_constraint = [1, 2, 3, 4, 5]
# dropout_rate = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
hidden_neuron = [1, 10, 20, 50, 70, 90, 100, 150, 200, 250, 300, 500, 1000]

param_distributions = dict(hidden_neuron = hidden_neuron)

search = RandomizedSearchCV(estimator=model, param_distributions=param_distributions, n_jobs=1, cv=3)
search = search.fit(X, y)

# summarize results
print("Best: %f using %s" % (search.best_score_, search.best_params_))
means = search.cv_results_['mean_test_score']
stds = search.cv_results_['std_test_score']
params = search.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))
    
ex_time = time.time() - start_time
print("\n--- %s minutes %d seconds ---\n" % (int(ex_time/60), int(ex_time%60)))

C:\Users\erwin\.conda\envs\ml-env\lib\site-packages\sklearn\model_selection\_search.py:285: UserWarning: The total space of parameters 2 is smaller than n_iter=10. Running 2 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best: 0.630360 using {'hidden_neuron': 10}
0.617260 (0.001629) with: {'hidden_neuron': 1}
0.630360 (0.000702) with: {'hidden_neuron': 10}

--- 4 minutes 57 seconds ---

